<a href="https://colab.research.google.com/github/TnzTanim/Android-App-TFlite-object-detection-/blob/main/Tflite_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating a Virtual Environment To install Model Maker

In [ ]:
%env PYTHONPATH = # /env/python

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages')

In [ ]:

!conda create -n myenv python=3.9

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install tflite-model-maker

In [ ]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install ipykernel

##Installing some additional packages

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install opencv-python
python --version


In [ ]:

%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install numpy==1.23.4

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
pip install pycocotools

#Training script

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
python Train.py

##Testing the model with sample images

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
from google.colab.patches import cv2_imshow

# Load the TensorFlow Lite model
model_path = '/content/Footballmodel.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Load the labels
classes = ['player', 'goalkeeper', 'ball', 'referee']  # Replace with actual class names

def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = img.resize((input_width, input_height))
    img = np.array(img, dtype=np.uint8)  # Change data type to UINT8
    return img

# Set the input image size based on the model's input dimensions
input_height, input_width = 320, 320

def detect_objects(interpreter, image, threshold):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()

    detection_boxes = interpreter.get_tensor(output_details[1]['index'])
    detection_scores = interpreter.get_tensor(output_details[0]['index'])  # Adjust this index as needed

    detection_boxes = np.squeeze(detection_boxes)
    detection_scores = np.squeeze(detection_scores)

    results = []
    for i in range(len(detection_scores)):
        if detection_scores[i] >= threshold:
            result = {
                'bounding_box': detection_boxes[i],
                'class_id': 0,  # Scalar value for detection_classes
                'score': detection_scores[i]
            }
            results.append(result)
    return results

# Run object detection and draw results
image_path = '/content/5.jpg'  # Replace with your image path
input_image = preprocess_image(image_path)
detection_results = detect_objects(interpreter, input_image[tf.newaxis, ...], threshold=0.1)

image_np = cv2.imread(image_path)
for obj in detection_results:
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * image_np.shape[1])
    xmax = int(xmax * image_np.shape[1])
    ymin = int(ymin * image_np.shape[0])
    ymax = int(ymax * image_np.shape[0])

    class_id = obj['class_id']
    class_name = classes[class_id]

    cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    label = f"{class_name}: {int(obj['score'] * 100)}%"
    cv2.putText(image_np, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2_imshow(image_np)

